In [2]:
import pandas as pd
import json
import time
from geopy.exc import GeocoderUnavailable, GeocoderTimedOut, GeocoderServiceError
from geopy.geocoders import Nominatim

rentdata_df = pd.read_json('rent_data.json')

In [3]:
rentdata_df

,title,address,price,area,type,description,bedroom,toilet,direction,law,floor,width,type_estate,image_links
0,"Cho thuê nhiều căn hộ Sunrise City, Sunrise Ci...","Sunrise City, 2325,27,33, Đường Nguyễn Hữu Thọ...",Thỏa thuận,80 m²,Cho Thuê,"* Nội dung.<br>Địa chỉ căn hộ ở 23,25,27,33 Đư...",2 phòng,2 phòng,null,null,null,null,Cho thuê căn hộ chung cư,https://file4.batdongsan.com.vn/resize/1275x71...
1,"Cho thuê căn hộ Vinhomes Central Park 1,2,3,4P...","Vinhomes Central Park, 720A, Đường Điện Biên P...",18 triệu/tháng,92 m²,Cho Thuê,Kính chào quý khách hàng! Xin gửi quý khách hà...,2 phòng,2 phòng,null,Sổ đỏ/ Sổ hồng,null,null,Cho thuê căn hộ chung cư,https://file4.batdongsan.com.vn/resize/1275x71...
2,"Opal Skyline cho thuê Studio 3tr5, 1PN 4tr, 2p...","Dự án Opal Skyline, Đường Nguyễn Văn Tiết, Phư...","3,5 triệu/tháng",40 m²,Cho Thuê,"Trung cho thuê căn hộ Opal Skyline, Lái Thiêu,...",1 phòng,1 phòng,Đông - Bắc,null,null,null,Cho thuê căn hộ chung cư,https://file4.batdongsan.com.vn/resize/1275x71...
3,"Chuyên cho thuê Vinhomes Grand Park, cập nhật ...","Dự án Vinhomes Grand Park, Phường Long Thạnh M...","4,5 triệu/tháng",30 m²,Cho Thuê,Chuyên cho thuê căn hộ giá Vinhomes Grand Park...,1 phòng,1 phòng,null,null,null,null,Cho thuê căn hộ chung cư,https://i1.ytimg.com/vi/zL2yWFcedao/hqdefault.jpg
4,Cho thuê văn phòng Tố Hữu Hà Đông DT 150m2 giá...,"Đường Tố Hữu, Phường Vạn Phúc, Hà Đông, Hà Nội",25 triệu/tháng,150 m²,Cho Thuê,Chủ đầu tư cho thuê văn phòng chia sẻ tầng 14 ...,null,null,Tây - Nam,null,null,null,Cho thuê văn phòng,https://file4.batdongsan.com.vn/resize/1275x71...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75,PKD New City cho thuê căn hộ từ 1 2 3PN. Đa dạ...,"New City Thủ Thiêm, 17, Đường Mai Chí Thọ, Phư...",11 triệu/tháng,51 m²,Cho Thuê,Phòng kinh doanh dự án New City chuyên về cho ...,1 phòng,1 phòng,null,null,null,null,Cho thuê căn hộ chung cư,https://file4.batdongsan.com.vn/resize/1275x71...
76,"Cho thuê nhà nguyên căn đường Thất Tùng Tùng, ...","Đường Tôn Thất Tùng, Phường Khương Thượng, Đốn...",9 triệu/tháng,40 m²,Cho Thuê,Cho thuê nhà nguyên căn gần đường Trường Chinh...,3 phòng,2 phòng,null,null,3 tầng,null,Cho thuê nhà riêng,https://file4.batdongsan.com.vn/resize/1275x71...
77,Cho thuê 2PN view sông view pháo hoa nội thất ...,"Dự án Empire City Thủ Thiêm, Đường Mai Chí Thọ...",36 triệu/tháng,98 m²,Cho Thuê,Cho thuê 2PN view hoa hậu Empire City.<br>- Di...,2 phòng,2 phòng,null,null,null,null,Cho thuê căn hộ chung cư,https://file4.batdongsan.com.vn/resize/1275x71...
78,Chính chủ cho thuê văn phòng dt 60 - 70m2 tại ...,"21, Đường Lê Văn Lương, Phường Nhân Chính, Tha...",12 triệu/tháng,60 m²,Cho Thuê,Tôi còn 2 sàn văn phòng tại nhà 21 Lê Văn Lươn...,null,null,null,null,null,null,Cho thuê văn phòng,https://file4.batdongsan.com.vn/resize/1275x71...


In [4]:
title = []                  # 1
price = []                  # 2
area = []                   # 3
bedroom = []        # 4
description = []            # 5
width = []                  # 6
direction = []              # 7
toilet = []         # 8
address = []               # 9
geo_code = []               # 10
law = []
floor = []
district = []
province_city = []
type = []
type_estate = []
image_links = []
phonenumber = []

In [5]:
import requests
import pandas as pd
import time

# Assuming rentdata_df is already defined and populated with data
rentdata_df["geo_code"] = [None for i in range(len(rentdata_df))]

def write_data(df, field, data_list):
    for i in df[field]:
        data_list.append(i)
        
def get_geocode(address):
    api_key = 'b48b7a0d92434d7ba9e13d0feb2db0ad'
    url = f"https://api.opencagedata.com/geocode/v1/json?q={address}&key={api_key}"

    response = requests.get(url)
    data = response.json()

    if data['results']:
        latitude = data['results'][0]['geometry']['lat']
        longitude = data['results'][0]['geometry']['lng']
        print(f"Latitude: {latitude}, Longitude: {longitude}")
        return [latitude, longitude]
    else:
        # Check if address contains "Đường"        
        if "Đường" in address:
            # Remove "Đường" and retry
            modified_address = address.replace("Đường ", "").strip()
            print(f"Retry with modified address: {modified_address}")
            return get_geocode(modified_address)  
        elif "Dự án" in address:
            # Remove "Đường" and retry
            modified_address = address.replace("Dự án ", "").strip()
            print(f"Retry with modified address: {modified_address}")
            return get_geocode(modified_address)      
        else:
            print("No results found.")
            return None

def data_normalization(df):
    for i, address in enumerate(df["address"]):
        df.at[i, "geo_code"] = get_geocode(address)
        district, province_city = extract_district_and_province(address)
        df.at[i, "district"] = district.replace('Huyện ', '').replace('Quận ', '').strip()            
        df.at[i, "province_city"] = province_city.replace('.','').strip()
        
    for i, price in enumerate(df['price']):
        if isinstance(price, str):           
            df.at[i, 'price'] = price.replace(' triệu/tháng', '').replace(',', '.').strip()    
            
    for i, toilet in enumerate(df['toilet']):
        if isinstance(toilet, str):
         df.at[i, 'toilet'] = toilet.replace(' phòng', '').strip()
        
    for i, bedroom in enumerate(df['bedroom']):        
        if isinstance(bedroom, str):
            df.at[i, 'bedroom'] = bedroom.replace(' phòng', '').strip()
    
    for i, area in enumerate(df['area']):
        if isinstance(area, str):
            df.at[i, 'area'] = area.replace(' m', '').replace('²', '').replace(',', '.').strip()    
    
    for i, width in enumerate(df['width']):        
        if isinstance(width, str):
            df.at[i, 'width'] = width.replace(' m', '').strip() 
                  
    for i, floor in enumerate(df['floor']):        
        if isinstance(floor, str):
            df.at[i, 'floor'] = floor.replace(' tầng', '').strip() 
            
    for i,type_estate in enumerate(df['type_estate']):        
        if isinstance(type_estate, str):
            df.at[i, 'type_estate'] = type_estate.replace('Cho thuê ', '').replace('Cho thuê, sang nhượng ', '').strip()

    for i, phonenumber in enumerate(df['phonenumber']):        
        if isinstance(phonenumber, str):
            phonenumber = phonenumber.replace(' ', '').replace('.', '').strip()
            # Kiểm tra nếu số đầu tiên khác 0 thì thêm số 0 vào đầu
            if phonenumber[0] != '0':
                phonenumber = '0' + phonenumber
            df.at[i, 'phonenumber'] = phonenumber.strip()
            
def extract_district_and_province(address):
    parts = address.split(',')
    if len(parts) >= 2:
        district = parts[-2].strip()
        province_city = parts[-1].strip()
        return district, province_city
    return None, None
        
data_normalization(rentdata_df)

rentdata_df['type_estate'] = rentdata_df['type_estate'].replace({
    'nhà mặt phố': 'Nhà Mặt Phố',
    'nhà biệt thự, liền kề': 'Nhà Biệt Thự, Liền Kề',
    'nhà riêng': 'Nhà Đất Thổ Cư',
    'căn hộ chung cư': 'Căn Hộ Chung Cư',
    'căn hộ chung cư mini': 'Căn Hộ Chung Cư',
    'đất nền dự án': 'Đất Nền Dự Án',
    'đất': 'Đất Nền Dự Án',
    'trang trại, khu nghỉ dưỡng': 'Đất Trang Trại',
    'condotel': 'Condotel',
    'văn phòng': 'Văn phòng, TTTM, Cửa hàng, Kiot',
    'shophouse, nhà phố thương mại': 'ShopHouse',
    'cửa hàng, ki ốt': 'Văn phòng, TTTM, Cửa hàng, Kiot',
    'kho, nhà xưởng, đất': 'Nhà Xưởng, Mặt Bằng, Kho Bãi',
    'kho, nhà xưởng': 'Nhà Xưởng, Mặt Bằng, Kho Bãi',
    'nhà trọ, phòng trọ': 'Nhà Trọ, Phòng Trọ',
})

write_data(rentdata_df, "title", title)
write_data(rentdata_df, "price", price)
write_data(rentdata_df, "area", area)
write_data(rentdata_df, "type", type)
write_data(rentdata_df, "type_estate", type_estate)
write_data(rentdata_df, "image_links", image_links)
write_data(rentdata_df, "bedroom", bedroom)
write_data(rentdata_df, "description", description)
write_data(rentdata_df, "width", width)
write_data(rentdata_df, "direction", direction)
write_data(rentdata_df, "toilet", toilet)
write_data(rentdata_df, "address", address)
write_data(rentdata_df, "law", law)
write_data(rentdata_df, "floor", floor)
write_data(rentdata_df, "geo_code", geo_code)
write_data(rentdata_df, "district", district)
write_data(rentdata_df, "province_city", province_city)
write_data(rentdata_df, "phonenumber", phonenumber)


Retry with modified address: Sunrise City, 2325,27,33, Nguyễn Hữu Thọ, Phường Tân Hưng, Quận 7, Hồ Chí Minh
Latitude: 10.18362, Longitude: 105.55522
Latitude: 10.79293, Longitude: 106.71852
Retry with modified address: Dự án Opal Skyline, Nguyễn Văn Tiết, Phường Lái Thiêu, Thuận An, Bình Dương
Latitude: 10.90689, Longitude: 106.70411
Retry with modified address: Vinhomes Grand Park, Phường Long Thạnh Mỹ, Quận 9, Hồ Chí Minh
No results found.
Latitude: 20.97136, Longitude: 105.77876
Retry with modified address: Vinhomes Grand Park, Phường Long Thạnh Mỹ, Quận 9, Hồ Chí Minh
No results found.
Retry with modified address: Vinhomes Grand Park, Phường Long Thạnh Mỹ, Quận 9, Hồ Chí Minh
No results found.
Latitude: 51.5, Longitude: 10.5
Latitude: 21.038799, Longitude: 105.791864
Retry with modified address: Dự án Chung cư CT3 Nghĩa Đô, Hoàng Quốc Việt, Phường Cổ Nhuế 2, Bắc Từ Liêm, Hà Nội
Retry with modified address: Chung cư CT3 Nghĩa Đô, Hoàng Quốc Việt, Phường Cổ Nhuế 2, Bắc Từ Liêm, Hà Nộ

In [6]:
rentdata_df

,title,address,price,area,type,description,bedroom,toilet,direction,law,floor,width,type_estate,image_links,geo_code,district,province_city
0,"Cho thuê nhiều căn hộ Sunrise City, Sunrise Ci...","Sunrise City, 2325,27,33, Đường Nguyễn Hữu Thọ...",Thỏa thuận,80,Cho Thuê,"* Nội dung.<br>Địa chỉ căn hộ ở 23,25,27,33 Đư...",2,2,null,null,null,null,Căn Hộ Chung Cư,https://file4.batdongsan.com.vn/resize/1275x71...,"[10.18362, 105.55522]",7,Hồ Chí Minh
1,"Cho thuê căn hộ Vinhomes Central Park 1,2,3,4P...","Vinhomes Central Park, 720A, Đường Điện Biên P...",18,92,Cho Thuê,Kính chào quý khách hàng! Xin gửi quý khách hà...,2,2,null,Sổ đỏ/ Sổ hồng,null,null,Căn Hộ Chung Cư,https://file4.batdongsan.com.vn/resize/1275x71...,"[10.79293, 106.71852]",Bình Thạnh,Hồ Chí Minh
2,"Opal Skyline cho thuê Studio 3tr5, 1PN 4tr, 2p...","Dự án Opal Skyline, Đường Nguyễn Văn Tiết, Phư...",3.5,40,Cho Thuê,"Trung cho thuê căn hộ Opal Skyline, Lái Thiêu,...",1,1,Đông - Bắc,null,null,null,Căn Hộ Chung Cư,https://file4.batdongsan.com.vn/resize/1275x71...,"[10.90689, 106.70411]",Thuận An,Bình Dương
3,"Chuyên cho thuê Vinhomes Grand Park, cập nhật ...","Dự án Vinhomes Grand Park, Phường Long Thạnh M...",4.5,30,Cho Thuê,Chuyên cho thuê căn hộ giá Vinhomes Grand Park...,1,1,null,null,null,null,Căn Hộ Chung Cư,https://i1.ytimg.com/vi/zL2yWFcedao/hqdefault.jpg,None,9,Hồ Chí Minh
4,Cho thuê văn phòng Tố Hữu Hà Đông DT 150m2 giá...,"Đường Tố Hữu, Phường Vạn Phúc, Hà Đông, Hà Nội",25,150,Cho Thuê,Chủ đầu tư cho thuê văn phòng chia sẻ tầng 14 ...,null,null,Tây - Nam,null,null,null,"Văn phòng, TTTM, Cửa hàng, Kiot",https://file4.batdongsan.com.vn/resize/1275x71...,"[20.97136, 105.77876]",Hà Đông,Hà Nội
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75,PKD New City cho thuê căn hộ từ 1 2 3PN. Đa dạ...,"New City Thủ Thiêm, 17, Đường Mai Chí Thọ, Phư...",11,51,Cho Thuê,Phòng kinh doanh dự án New City chuyên về cho ...,1,1,null,null,null,null,Căn Hộ Chung Cư,https://file4.batdongsan.com.vn/resize/1275x71...,"[10.39426, 105.40751]",2,Hồ Chí Minh
76,"Cho thuê nhà nguyên căn đường Thất Tùng Tùng, ...","Đường Tôn Thất Tùng, Phường Khương Thượng, Đốn...",9,40,Cho Thuê,Cho thuê nhà nguyên căn gần đường Trường Chinh...,3,2,null,null,3,null,Nhà Đất Thổ Cư,https://file4.batdongsan.com.vn/resize/1275x71...,"[21.0023482, 105.8296355]",Đống Đa,Hà Nội
77,Cho thuê 2PN view sông view pháo hoa nội thất ...,"Dự án Empire City Thủ Thiêm, Đường Mai Chí Thọ...",36,98,Cho Thuê,Cho thuê 2PN view hoa hậu Empire City.<br>- Di...,2,2,null,null,null,null,Căn Hộ Chung Cư,https://file4.batdongsan.com.vn/resize/1275x71...,None,2,Hồ Chí Minh
78,Chính chủ cho thuê văn phòng dt 60 - 70m2 tại ...,"21, Đường Lê Văn Lương, Phường Nhân Chính, Tha...",12,60,Cho Thuê,Tôi còn 2 sàn văn phòng tại nhà 21 Lê Văn Lươn...,null,null,null,null,null,null,"Văn phòng, TTTM, Cửa hàng, Kiot",https://file4.batdongsan.com.vn/resize/1275x71...,"[14.48985, 109.09162]",Thanh Xuân,Hà Nội


In [7]:
def evaluate_geocode_accuracy(df):
    total_addresses = len(df)
    successful_geocodes = df['geo_code'].count()

    success_rate = (successful_geocodes / total_addresses) * 100
    failure_rate = 100 - success_rate

    print(f"Total addresses: {total_addresses}")
    print(f"Successful geocodes: {successful_geocodes} addresses ({success_rate:.2f}% success)")
    print(f"Failed geocodes: {total_addresses - successful_geocodes} addresses ({failure_rate:.2f}% failure)")

# Evaluate geocoding accuracy for your DataFrame
evaluate_geocode_accuracy(rentdata_df)

Total addresses: 80
Successful geocodes: 72 addresses (90.00% success)
Failed geocodes: 8 addresses (10.00% failure)


In [8]:
rentdata_df.to_csv('rent_data.csv', encoding='utf-8', index=False)

In [9]:
import pymysql

# Thiết lập kết nối tới cơ sở dữ liệu MySQL
connection = pymysql.connect(
    host='localhost',
    user='root',
    password='123456',
    database='estate',
    charset='utf8mb4',
    cursorclass=pymysql.cursors.DictCursor
)

try:
    with connection.cursor() as cursor:
        # Xóa dữ liệu từ cả hai bảng pools_bds và pools_image
        cursor.execute("""
            DELETE pb, pi
            FROM pools_bds pb
            JOIN pools_image pi ON pb.BDS_id = pi.BDS_id
            WHERE pb.kind = 'Cho Thuê' AND (pi.content LIKE '%tdongsan.com%' OR pi.content LIKE '%ytimg.com%')
        """)
        connection.commit()

finally:
    connection.close()


In [10]:
import csv
import pymysql

# Thiết lập kết nối tới cơ sở dữ liệu MySQL với mã hóa utf8mb4
connection = pymysql.connect(
    host='localhost',
    user='root',
    password='123456',
    database='estate',
    charset='utf8mb4',
    cursorclass=pymysql.cursors.DictCursor
)

def is_decimal(value):
    try:
        float(value)
        return True
    except ValueError:
        return False

try:
    with connection.cursor() as cursor:
        
        # Đọc dữ liệu từ file CSV và nhập vào MySQL
        with open('rent_data.csv', 'r', encoding='utf-8') as file:
            csv_data = csv.DictReader(file)
            for row in csv_data:
                try:
                    district = row['district']
                    province_city = row['province_city']
                    
                    # Kiểm tra tồn tại address
                    cursor.execute("SELECT address_id FROM pools_address WHERE district = %s AND province_city = %s", (district, province_city))
                    existing_address = cursor.fetchone()
                    if existing_address:
                        address_id = existing_address['address_id']
                    else:
                        # Nếu không tìm thấy địa chỉ, bỏ qua dòng này
                        print(f"Không tìm thấy địa chỉ cho dòng: {row}")
                        continue
                    
                    # Kiểm tra pools_typeofbds 
                    bds_type = row['type_estate']
                    cursor.execute("SELECT Type_Id FROM pools_typeofbds WHERE TypeName = %s", (bds_type,))
                    existing_type = cursor.fetchone()
                    if existing_type:
                        type_id = existing_type['Type_Id']
                    else:
                        # Nếu không tìm thấy loại bất động sản, bỏ qua dòng này
                        print(f"Không tìm thấy loại bất động sản cho dòng: {row}")
                        continue
                    
                    # Thêm dữ liệu vào bảng pools_bds
                    title = row['title']
                    price = row['price']
                    area = row['area']
                    kind = row['type']
                    law = row['law']
                    bedroom = row['bedroom']
                    floor = row['floor']
                    description = row['description']
                    width = row['width']
                    direction = row['direction']
                    toilet = row['toilet']
                    location = row['address']
                    geocode = row['geo_code']
                    
                    # Handle 'price' column
                    if price.strip().lower() == 'thỏa thuận':
                        price = 0  # or set to a default value, like 0
                    elif not is_decimal(price):
                        price = 0  # or set to a default value

                    # Handle 'width' column
                    if not is_decimal(width):
                        width = 0  # or set to a default value

                    # Handle 'bedroom' column
                    if not bedroom.strip() or bedroom.lower() == 'null':
                        bedroom = 0  # or set to a default value
                       
                    # Handle 'floor' column 
                    if not floor.strip() or floor.lower() == 'null':
                        floor = 0  # or set to a default value

                    # Handle 'toilet' column (if needed)
                    if not is_decimal(toilet):
                        toilet = 0  # or set to a default value

                    # Kiểm tra geocode nếu là None hoặc rỗng, bỏ qua dòng này
                    if not geocode:
                        # print(f"Không có giá trị geocode cho dòng: {row}")
                        continue

                    # Chuyển geo_code thành string nếu nó là list
                    if isinstance(geocode, list):
                        geocode = str(geocode)

                    # Thêm dữ liệu vào bảng pools_bds
                    cursor.execute("""
                        INSERT INTO pools_bds (title, price, area, kind, law, bedroom, floor, address_id, Type_id, description, width, direction, toilet, location, geocode)
                        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
                    """, (title, price, area, kind, law, bedroom, floor, address_id, type_id, description, width, direction, toilet, location, geocode))
                    connection.commit()
                    
                    # Lấy BDS_id của BDS vừa thêm
                    cursor.execute("SELECT LAST_INSERT_ID()")
                    bds_id = cursor.fetchone()['LAST_INSERT_ID()']
                    
                    # Thêm dữ liệu vào bảng pools_image (nếu có)
                    if 'image_links' in row and row['image_links']:  # Kiểm tra xem cột 'image_link' có tồn tại và có giá trị không
                        image_link = row['image_links']
                        cursor.execute("INSERT INTO pools_image (BDS_id, content) VALUES (%s, %s)", (bds_id, image_link))
                        connection.commit()
                        
                except Exception as e:
                    print(f"Lỗi khi xử lý dòng: {row}")
                    print(e)
finally:
    connection.close()


Không tìm thấy địa chỉ cho dòng: {'title': 'Sẵn giỏ hàng 300 căn Opal Skyline 1pn 2pn 3pn nhà mới, bao PQL, xem nhà liền. Call 0965 431 ***', 'address': 'Dự án Opal Skyline, Thành phố Thuận An, Bình Dương', 'price': '3', 'area': '40', 'type': 'Cho Thuê', 'description': '- Thiết kế: 1PN - 1WC - Diện tích: 40m² - Giá thuê: Từ 3.000.000 VND.<br>- Thiết kế: 2PN - 2WC - Diện tích: 65m² - Giá thuê: 4.500.000 VND.<br>- Thiết kế: 3PN - 2WC - Diện tích: 78m² - Giá thuê: 6.000.000 VND.<br>-----------------------------------------------<br>- Cư dân nơi đây có thể tận hưởng cuộc sống tiện nghi với những tiện ích ấn tượng ngay trong nội khu chung cư như: Công viên, khu vui chơi trẻ em, hồ bơi tràn viền, siêu thị, cửa hàng tiện lợi, quán cafe, nhà hàng, phòng gym, và trung tâm Anh ngữ.<br>- Khu nhà an toàn với sự giám sát 24/7 của đội bảo vệ chung cư và hệ thống camera an ninh hiện đại.<br>- Vị trí đắc địa, dễ dàng kết nối giao thông với các quận trung tâm khác.<br>- Tiện ích ngoại khu xung quanh dự